In [ ]:
pip install earthengine-api

In [ ]:
pip install earthengine-api --upgrade

In [ ]:
pip install earthpy

In [69]:
pip install geehydro

  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10143 sha256=e94dc2981f845c613499fb38efd57504750e96e1877fedcea15aa32bfe4e9f86
  Stored in directory: /root/.cache/pip/wheels/49/6c/92/019aeb37663078e6666afa9dd336e2d8df72d5683d7d357697
Successfully built geehydro


In [70]:
import ee
import datetime

import earthpy as ep

from glob import glob
import geehydro

import earthpy.spatial as es

import rasterio as rio
from rasterio.plot import plotting_extent
from rasterio.plot import show
from rasterio.plot import reshape_as_raster, reshape_as_image

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

import plotly.graph_objects as go


In [5]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=e_f521QUyEEiFrFlOFqGjenmQ8Wd4yJx8Iylg4f5FiA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiFQqfHhi38I4bu3NWaEQwuD6NpILp76Z1-xGcU5GbEIIx7oX0ir2c

Successfully saved authorization token.


In [207]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")


In [109]:
# the Ituna/Itatá Indigenous Land in Brazil.
Ituna_map = folium.Map(location=[-4.06738, -52.034], zoom_start= 15)
Ituna_map

In [198]:
# the Ituna/Itatá Indigenous Land in Brazil.
geo_map = folium.Map(location=[25.433457120928352, -80.92489760146748], zoom_start= 7)
geo_map
'''
geometry =  ee.Geometry.Polygon([[[-80.92489760146748, 25.433457120928352],
                                 [-80.64474623427998, 25.488013471687964],
                                 [-80.57882826552998, 25.710940372707608],
                                 [-81.02377455459248, 25.770317250349557],
                                 [-80.95236342177998, 25.552457242621447]]])
'''
geo_map

In [208]:
# setting the Area of Interest (AOI)
Ituna_AOI = ee.Geometry.Rectangle([-51.84448, -3.92180,
                                   -52.23999, -4.38201])


geometry = ee.Geometry.Rectangle([-112.125092,36.056595, 
                                 -113.125092,38.056595])

In [221]:
# filter area
landsat_AOI = landsat.filterBounds(Ituna_AOI).filterDate('2010-01-01','2021-12-31')
landsat_geo = landsat.filterBounds(geometry)

In [210]:
# choose dates
landsat = landsat.filterDate('2010-01-01','2021-12-31')

In [ ]:
landsat_AOI.getInfo()

In [222]:
# Check how many images you have
print('Total number:', landsat_AOI.size().getInfo())

# Check how many images you have
print('Total number:', landsat_geo.size().getInfo())


Total number: 116
Total number: 1144


In [213]:
# Names of Landsat 8 Bands
landsat_AOI.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

In [223]:
from datetime import datetime as dt

# the least cloudy image
least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())


# how cloudy is it?
print('Cloud Cover (%):', least_cloudy.get('CLOUD_COVER').getInfo())

# when was this image taken?
date = ee.Date(least_cloudy.get('system:time_start'))

time = date.getInfo()['value']/1000

dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

Cloud Cover (%): 0


'2017-07-20 13:36:07'

In [224]:
least_cloudy

In [225]:
parameters = {'min': 0,
              'max': 1000,
              'dimensions': 512,
              'bands': ['B4', 'B3', 'B2'],
              'region': Ituna_AOI}


imgdown = least_cloudy.toUint16()
             
Image(url = imgdown.getThumbUrl(parameters))

In [226]:
task = ee.batch.Export.image.toDrive(image=imgdown,
                                     description='Brazil Image',
                                     scale=30,
                                     fileNamePrefix='my_export_brazil',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
task.start()

In [138]:
task.status()

{'attempt': 1,
 'creation_timestamp_ms': 1646316347191,
 'description': 'Brazil Image',
 'destination_uris': ['https://drive.google.com/'],
 'id': 'OEBGWBXGXZKVMHX53B42PV3G',
 'name': 'projects/earthengine-legacy/operations/OEBGWBXGXZKVMHX53B42PV3G',
 'start_timestamp_ms': 1646316439157,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1646316914033}

# NVDI 
please oh god work

In [186]:
ndvi = least_cloudy.normalizedDifference(['B5', 'B4'])


In [227]:
palette = ['red', 'yellow', 'green']
ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': Ituna_AOI}
Ituna_map.addLayer(ndvi, ndvi_parameters)
Ituna_map

In [228]:
#NDVI calculation:
def ndvi_func(i):
    ndvi = i.normalizedDifference (['B5', 'B4']).rename('NDVI')
    return i.addBands(ndvi)

images_ndvi = landsat_AOI.map(ndvi_func)

#Calculating year wise NDVI.  Result is a list of dictionaries.
years = ee.List.sequence(2010,2021)
def year_func(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    mean = (images_ndvi
            .filter(range)
            .select('NDVI')
            .mean()
            .reduceRegion(ee.Reducer.mean(), geometry, 30))
    return mean.set('year', y)

yearwise_ndvi = years.map(year_func).getInfo()
df = pd.DataFrame(yearwise_ndvi)
print(df)

EEException: ignored

In [200]:
import ee
import pandas as pd

ee.Initialize()

l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
geometry =  ee.Geometry.Polygon([[[-80.92489760146748, 25.433457120928352],
                                 [-80.64474623427998, 25.488013471687964],
                                 [-80.57882826552998, 25.710940372707608],
                                 [-81.02377455459248, 25.770317250349557],
                                 [-80.95236342177998, 25.552457242621447]]])

#ee.Geometry.Rectangle([-112.125092,36.056595,  -113.125092, 37.056595])

#Filtering date, polygon, and cloudiness
images = (l8.filterDate('2010-09-01', '2021-12-31')
          .filterBounds(geometry)
          .filter('CLOUD_COVER < 5'))

#NDVI calculation:
def ndvi_func(i):
    ndvi = i.normalizedDifference (['B5', 'B4']).rename('NDVI')
    return i.addBands(ndvi)

images_ndvi = images.map(ndvi_func)

print(images_ndvi)

#Calculating year wise NDVI.  Result is a list of dictionaries.
years = ee.List.sequence(2010,2021)
def year_func(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    mean = (images_ndvi
            .filter(range)
            .select('NDVI')
            .mean()
            .reduceRegion(ee.Reducer.mean(), geometry, 30))
    return mean.set('year', y)

yearwise_ndvi = years.map(year_func).getInfo()
df = pd.DataFrame(yearwise_ndvi)
print(df)

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.map",
    "arguments": {
      "baseAlgorithm": {
        "functionDefinitionValue": {
          "argumentNames": [
            "_MAPPING_VAR_0_0"
          ],
          "body": {
            "functionInvocationValue": {
              "functionName": "Image.addBands",
              "arguments": {
                "dstImg": {
                  "argumentReference": "_MAPPING_VAR_0_0"
                },
                "srcImg": {
                  "functionInvocationValue": {
                    "functionName": "Image.rename",
                    "arguments": {
                      "input": {
                        "functionInvocationValue": {
                          "functionName": "Image.normalizedDifference",
                          "arguments": {
                            "bandNames": {
                              "constantValue": [
                                "B5",
                      

In [133]:
from collections import Counter

In [134]:
np.percentile(ndvi.data.flatten(), 100)

AttributeError: ignored

In [136]:
# Create classes and apply to NDVI results
ndvi_class_bins = [-np.inf, 0, 0.15, 0.23, 0.6, np.inf]


ndvi_landsat_class = np.digitize(ndvi, ndvi_class_bins)
print(ndvi_landsat_class)

# Apply the nodata mask to the newly classified NDVI data
ndvi_landsat_class = np.ma.masked_where(np.ma.getmask(ndvi), ndvi_landsat_class)

# Get counting of each class
counts = Counter(ndvi_landsat_class.data.flatten())

# Get the average
_sum = sum(counts.values())
for idx, val in counts.items():
  counts[idx] = np.round(val/_sum,3)

nbr_colors = ["purple", "y", "yellowgreen", "g", "darkgreen"]
nbr_cmap = ListedColormap(nbr_colors)

# Define class names
ndvi_cat_names = [
    "No Vegetation",
    "Bare Area",
    "Low Vegetation",
    "Moderate Vegetation",
    "High Vegetation",
]

for i in range(len(ndvi_cat_names)):
  ndvi_cat_names[i] += " " + str(counts[i+1]*100) + "%"

# Get list of classes
classes = np.unique(ndvi_landsat_class)

classes = classes.tolist()

# The mask returns a value of none in the classes. remove that
classes = classes[0:5]

# Plot the data
fig, ax = plt.subplots(figsize=(10, 10))

im = ax.imshow(ndvi_landsat_class, cmap=nbr_cmap)

epp.draw_legend(im_ax=im, classes=classes, titles=ndvi_cat_names)

ax.set_title(
    "Landsat 8 - Normalized Difference Vegetation Index (NDVI) Classes",
    fontsize=14,
)

ax.set_axis_off()

# Auto adjust subplot to fit figure size

plt.tight_layout()


TypeError: ignored

# Leftover Junk

In [76]:
# Interactive Map
Map = folium.Map(location=[-14.1862, 27.562], zoom_start=8)
Map.setOptions('SATELLITE')
Map

## Landsat Data

In [77]:
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
Zambia = countries.filter(ee.Filter.eq("country_na", "Zambia"))

dataset = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate('2017-01-01', '2017-12-31').filterBounds(Zambia)

trueColor432 = dataset.select(['B4', 'B3', 'B2'])
trueColor432Vis = {
    'min': 0.0,
    'max': 0.4,
}

Map.addLayer(trueColor432.median(), trueColor432Vis, 'True Color (432)')

#Use Map.addLayer() to add features and feature collections to the map
Map.addLayer(ee.Image().paint(Zambia, 0, 2), {}, 'Zambia')

## Display

In [78]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

In [92]:
from IPython.display import Image

#dataset = lst
dataset = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate('2017-01-01', '2017-12-31')

# Define the urban location of interest as a point near Lyon, France.
u_lon = 4.8148
u_lat = 45.7758
u_poi = ee.Geometry.Point(u_lon, u_lat)
scale = 1000  # scale in meters

# Define a region of interest with a buffer zone of 1000 km around Lyon.
roi = u_poi.buffer(1e6)

# Reduce the LST collection by mean.
lst_img = dataset.mean()

# Create a URL to the styled image for a region around France.
url = lst_img.getThumbUrl({
    'min': 10, 'max': 30, 'dimensions': 512, 'region': roi,
    'palette': ['blue', 'yellow', 'orange', 'red']})
print(url)

# Display the thumbnail land surface temperature in France.
print('\nPlease wait while the thumbnail loads, it may take a moment...')
Image(url=url)

EEException: ignored

In [ ]:
task = ee.batch.Export.image.toDrive(image=elv_img,
                                     description='elevation_near_lyon_france',
                                     scale=30,
                                     region=lyon,
                                     fileNamePrefix='my_export_lyon',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
task.start()

## Composite

In [90]:
composite = ee.Algorithms.Landsat.simpleComposite(collection=dataset,asFloat=True)

rgbVis = {'bands':['B4', 'B3', 'B2'], 'min': 0, 'max':0.3}
nirVis = {'bands': ['B5', 'B4', 'B3'], 'min':0, 'max': [0.5, 0.3, 0.3]}

In [91]:
# For Visualization
import folium


def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

# Add EE drawing method to Folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[9,39], zoom_start=15) #zoom influences how zoomed in you are to begin with, lower = farther away

# Add layer to map object
my_map.add_ee_layer(composite, rgbVis, 'RGB')
my_map.add_ee_layer(composite, nirVis, 'False COlor')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

EEException: ignored

In [63]:
#Landsat_composite in Dresden area
area_dresden = list([(13.6, 50.96), (13.9, 50.96), (13.9, 51.12), (13.6, 51.12), (13.6, 50.96)])
area_dresden = ee.Geometry.Polygon(area_dresden)
time_range_dresden = ['2010-07-28', '2010-08-05']

collection_dresden = ('LANDSAT/LE07/C02/T1_L2')
print(type(area_dresden))


<class 'ee.geometry.Geometry'>


In [64]:
import folium
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [67]:
Mapdisplay(center=center, dicc=area_dresden)

AttributeError: ignored

In [61]:
# Load a raw Landsat 5 ImageCollection for a single year.
collection = ee.ImageCollection('LANDSAT/LT05/C02/T1')

#adjustBands = collection.select(['SR_B7', 'SR_B5', 'SR_B4', 'SR_B3','SR_B2', 'SR_B1'],['B7', 'B5', 'B4', 'B3', 'B2', 'B1'])

In [62]:


# Create a cloud-free composite with default parameters.
composite = ee.Algorithms.Landsat.simpleComposite(collection)

# Create a cloud-free composite with custom parameters for
# cloud score threshold and percentile.
customComposite = ee.Algorithms.Landsat.simpleComposite(
  collection=collection,
  percentile=75,
  cloudScoreRange=5
)

# Display the composites.
center=[37.7726,-122.3578]

composite_tk=composite.getMapId({'bands': ['B1', 'B3', 'B2'], 'max': 128})
customComposite_tk = customComposite.getMapId({'bands':['B1', 'B3', 'B2'], 'max':128})

Mapdisplay(center=center,
           dicc={'TOA composite':composite_tk,'Custom TOA composite':customComposite_tk},
           zoom_start=10)

In [27]:
#Methods from climada.util.earth_engine module
def obtain_image_landsat_composite(collection, time_range, area):
    """ Selection of Landsat cloud-free composites in the Earth Engine library
    See also: https://developers.google.com/earth-engine/landsat

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        image_composite (ee.image.Image)
     """
    collection = ee.ImageCollection(collection)

    ## Filter by time range and location
    collection_time = collection.filterDate(time_range[0], time_range[1])
    image_area = collection_time.filterBounds(area)
    #image_composite = ee.Algorithms.Landsat.surfaceReflectance(image_area, 75, 3)
    image_composite = ee.Algorithms.Landsat.simpleComposite(image_area, 75, 3)
    return image_composite

def obtain_image_median(collection, time_range, area):
    """ Selection of median from a collection of images in the Earth Engine library
    See also: https://developers.google.com/earth-engine/reducers_image_collection

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        image_median (ee.image.Image)
     """
    collection = ee.ImageCollection(collection)

    ## Filter by time range and location
    collection_time = collection.filterDate(time_range[0], time_range[1])
    image_area = collection_time.filterBounds(area)
    image_median = image_area.median()
    return image_median

def obtain_image_sentinel(collection, time_range, area):
    """ Selection of median, cloud-free image from a collection of images in the Sentinel 2 dataset
    See also: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        sentinel_median (ee.image.Image)
     """
#First, method to remove cloud from the image
    def maskclouds(image):
        band_qa = image.select('QA60')
        cloud_mask = ee.Number(2).pow(10).int()
        cirrus_mask = ee.Number(2).pow(11).int()
        mask = band_qa.bitwiseAnd(cloud_mask).eq(0) and(
            band_qa.bitwiseAnd(cirrus_mask).eq(0))
        return image.updateMask(mask).divide(10000)

    sentinel_filtered = (ee.ImageCollection(collection).
                         filterBounds(area).
                         filterDate(time_range[0], time_range[1]).
                         filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).
                         map(maskclouds))

    sentinel_median = sentinel_filtered.median()
    return sentinel_median


In [28]:
#Application to examples
composite_dresden = obtain_image_landsat_composite(collection_dresden, time_range_dresden, area_dresden)
print(composite_dresden.getInfo())

EEException: ignored

In [22]:
def get_region(geom):
    """Get the region of a given geometry, needed for exporting tasks.

    Parameters:
        geom (ee.Geometry, ee.Feature, ee.Image): region of interest

    Returns:
        region (list)
    """
    if isinstance(geom, ee.Geometry):
        region = geom.getInfo()["coordinates"]
    elif isinstance(geom, ee.Feature, ee.Image):
        region = geom.geometry().getInfo()["coordinates"]
    elif isinstance(geom, list):
        condition = all([isinstance(item) == list for item in geom])
        if condition:
            region = geom
    return region


region_dresden = get_region(area_dresden)


In [ ]:
import webbrowser
def get_url(name, image, scale, region):
    """It will open and download automatically a zip folder containing Geotiff data of 'image'.
    If additional parameters are needed, see also:
    https://github.com/google/earthengine-api/blob/master/python/ee/image.py

    Parameters:
        name (str): name of the created folder
        image (ee.image.Image): image to export
        scale (int): resolution of export in meters (e.g: 30 for Landsat)
        region (list): region of interest

    Returns:
        path (str)
     """
    path = image.getDownloadURL({
        'name':(name),
        'scale': scale,
        'region':(region)
        })

    webbrowser.open_new_tab(path)
    return path



#url_swiss = get_url('swiss_pop', median_swiss, 900, region_swiss)
url_dresden = get_url('dresden', composite_dresden, 30, region_dresden)
#url_landcover = get_url('landcover_swiss', landCover_layer, 100, region_swiss)

#For the example of Zürich, due to size, it doesn't work on Jupyter Notebook but it works on Python
#url_zurich = get_url('sentinel', zurich_band, 10, region_zurich)

print(url_swiss)
print(url_dresden)
print(url_landcover)